# Agentic Learning Platform - Database Analysis

This notebook provides tools for exploring and analyzing student performance data from the learning platform.

## Database Schema

- **students**: Student profiles (name, grade_level)
- **sessions**: Learning sessions (start_time, end_time, module_id)
- **activity_attempts**: Quiz attempts (score, total, difficulty, item_results)
- **chat_messages**: Conversation history between student and agent

## Setup

In [1]:
# Import libraries
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
from pathlib import Path

# Set up plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Database path

DB_PATH = 'learning.db'

print(f"Database: {DB_PATH}")
print(f"Exists: {Path(DB_PATH).exists()}")

Database: learning.db
Exists: True


## Database Connection

In [2]:
# Connect to database
conn = sqlite3.connect(DB_PATH)

# Helper function to run queries
def query(sql, params=None):
    """Execute SQL query and return DataFrame"""
    if params:
        
        return pd.read_sql_query(sql, conn, params=params)
    return pd.read_sql_query(sql, conn)

print("✓ Connected to database")

✓ Connected to database


## 1. Basic Data Exploration

### View All Tables

In [3]:
# List all tables
tables = query("""
    SELECT name FROM sqlite_master 
    WHERE type='table'
    ORDER BY name
""")

print("Tables in database:")
display(tables)

Tables in database:


,name
0,activity_attempts
1,chat_messages
2,sessions
3,student_proficiencies
4,students


In [7]:
profs = query("""
SELECT * from students 
where name ="Bob";
""")

display(profs)

,student_id,name,grade_level,created_at
0,140d959c-19a4-4d5d-81af-701a279b8aaf,Bob,3,2025-10-31 21:19:18.556259


In [61]:
profs = query("""
---SELECT * from students where student_id ="140d959c-19a4-4d5d-81af-701a279b8aaf";
---SELECT * from sessions where student_id ="140d959c-19a4-4d5d-81af-701a279b8aaf";
---SELECT * from activity_attempts where student_id ="140d959c-19a4-4d5d-81af-701a279b8aaf";
SELECT * from student_proficiencies where student_id ="140d959c-19a4-4d5d-81af-701a279b8aaf";
""")

display(profs)

,proficiency_id,student_id,level,domain,module_id,item_id,alpha,beta,mean_ability,confidence,learning_rate,forgetting_rate,sample_count,last_updated,created_at
0,ed6e47bb-7dd1-484c-b56e-c39f7879c47e,140d959c-19a4-4d5d-81af-701a279b8aaf,domain,reading,None,None,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581868
1,e199ff30-9a46-4330-8817-26ff3ff82d45,140d959c-19a4-4d5d-81af-701a279b8aaf,module,reading,r003.1,None,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581877
2,f8c8c545-adcd-40ac-b35c-ffe2a019c83d,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,pirate,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581884
3,2f5d4b88-c504-4b0b-a667-dd3d75a45050,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,parrot,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581891
4,292c46c3-1ec1-43f0-b1da-599ca29f9199,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,ship,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581898
5,15e2e098-f6d5-4d8d-bd36-a8c6f1a8bc61,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,shape,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581905
6,7e2f469f-566c-4f54-9cba-4a29ea9633a1,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,grog,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581911
7,5a9ba6b1-3d59-44c2-bd8a-3f52b600ff6c,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,key,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581918
8,ea15d8ee-ce8b-49a5-ae8e-81900bbcd616,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,door,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581924
9,88f46466-6173-4255-9b9d-9938da542461,140d959c-19a4-4d5d-81af-701a279b8aaf,item,reading,r003.1,chest,1.0,1.0,0.5,0.0,0.1,0.05,0,2025-10-31 23:34:23,2025-10-31 21:19:18.581931


In [55]:
results = query(
"""
SELECT 
    date,
    score,
    total,
    json_array_length(item_results) as num_items,
    json_extract(item_results, '$[0]') as first_item,
    json_extract(item_results, '$[1]') as second_item
FROM activity_attempts
WHERE student_id = '140d959c-19a4-4d5d-81af-701a279b8aaf'
ORDER BY date DESC
LIMIT 1;
""")

display(results)
results.to_csv('my_data.csv', index=False)
#row_dict = results.iloc[0].to_dict()
#print(json.dumps(row_dict, indent=2, default=str))

,date,score,total,num_items,first_item,second_item
0,2025-10-31 23:23:34.838371,10,10,10,"{""questionNumber"":1,""definition"":""you can eith...","{""questionNumber"":2,""definition"":""a strong dri..."


In [59]:
act = query("""
   UPDATE student_proficiencies
SET 
    alpha = 1.0,
    beta = 1.0,
    mean_ability = 0.5,
    confidence = 0.0,
    sample_count = 0,
    last_updated = CURRENT_TIMESTAMP
WHERE student_id = '140d959c-19a4-4d5d-81af-701a279b8aaf';
""")



TypeError: 'NoneType' object is not iterable

### Students

In [6]:
students = query("""
    SELECT *
    FROM students st
""")
display(students)

,student_id,name,grade_level,created_at
0,b79bfa1c-20eb-483a-b5bb-b5d322281a58,BRad,3,2025-10-17 23:14:07.539417
1,57f0974d-85ac-41cf-8a20-8cbb3c6a7456,brad,3,2025-10-17 23:14:52.936853
2,fee2dfa9-b73f-47e6-a6e4-eeb6fe89f5a8,Brad,3,2025-10-17 23:50:16.035281
3,fa3b0827-a922-42bb-a6bb-66e02bf464b0,brad,3,2025-10-18 00:14:20.959086
4,56afb2fb-fa85-4676-bb37-367b11941387,brad,3,2025-10-18 00:16:26.799407
5,7defd540-f064-4c80-8ef7-d77e0ec7e0d8,BRad,3,2025-10-18 00:20:01.434088
6,3d7b25e8-ff0e-42f8-b16f-9de1d01be4d6,Brad,3,2025-10-21 20:51:50.243243
7,36ee3470-e3f1-4cd2-b152-535d6d83fea3,quite(),3,2025-10-21 21:07:47.827715
8,253e1df8-2bed-4f79-bafe-c0dfa2acf18e,,3,2025-10-21 21:08:02.291332
9,b14bb1cc-3925-4eee-b561-307e20bc6d90,Folioso,3,2025-10-21 22:34:19.234283


In [7]:
# View all students
students = query("""
    SELECT 
        st.student_id,
        st.name,
        st.grade_level,
        st.created_at,
        COUNT(DISTINCT s.session_id) as total_sessions,
        COUNT(DISTINCT a.attempt_id) as total_attempts
    FROM students st
    LEFT JOIN sessions s ON st.student_id = s.student_id
    LEFT JOIN activity_attempts a ON st.student_id = a.student_id
    GROUP BY st.student_id
    ORDER BY created_at DESC
""")

print(f"Total Students: {len(students)}")
display(students)

Total Students: 32


,student_id,name,grade_level,created_at,total_sessions,total_attempts
0,140d959c-19a4-4d5d-81af-701a279b8aaf,Bob,3,2025-10-31 21:19:18.556259,1,0
1,12bf95dc-413b-4541-9e15-531b387d8e61,Wenceslas,3,2025-10-30 22:06:47.993785,1,0
2,8446973b-2c4d-440e-9e2e-5377824eff4a,Paul,3,2025-10-30 22:00:47.742659,2,0
3,fe5628eb-e9d5-4205-b5dd-077a25ff06b2,George,3,2025-10-30 02:28:28.964843,2,0
4,e1f08120-6c19-4fea-baaa-f1154a207c66,arwtgqhw,3,2025-10-22 01:37:45.417801,1,0
5,3c1ec143-b7d0-4e57-ac31-3002e254813b,arwtgqhw,3,2025-10-22 01:37:40.572507,1,0
6,9a15c4e2-04ab-467c-8ef5-f65d651c5fed,arwtgqhw,3,2025-10-22 01:27:22.933234,1,0
7,15e96fe0-796d-4090-8b31-76ec86d3af7f,erwgewrt,3,2025-10-22 01:27:13.938367,1,0
8,bf4407e0-36c8-455f-b534-68737ab45bbf,erwgewrt,3,2025-10-22 01:26:26.039572,1,0
9,055d5d5e-8f69-45dc-a085-673a7d87506b,erwgewrt,3,2025-10-22 01:23:26.758618,1,0


### Sessions

In [8]:
# View all sessions with student names
sessions = query("""
    SELECT 
        s.session_id,
        st.name as student_name,
        s.module_id,
        s.start_time,
        s.end_time,
        COUNT(a.attempt_id) as num_attempts
    FROM sessions s
    JOIN students st ON s.student_id = st.student_id
    LEFT JOIN activity_attempts a ON s.session_id = a.session_id
    GROUP BY s.session_id
    ORDER BY s.start_time DESC
""")

print(f"Total Sessions: {len(sessions)}")
display(sessions.head(10))

Total Sessions: 74


,session_id,student_name,module_id,start_time,end_time,num_attempts
0,7c944f19-d3dc-4136-af7a-c3ac4485bb1d,Bob,r003.1,2025-10-31 21:19:18.562194,None,0
1,18c836a9-5e81-448a-aaf4-81743d4997da,Brad,r003.1,2025-10-30 23:36:59.906718,None,0
2,fdf557d3-d07c-40d4-9866-d002aca43352,Brad,r003.1,2025-10-30 23:36:56.591151,None,0
3,11e31e89-4e2b-4448-a971-a132f3c9a7e8,Wenceslas,r003.1,2025-10-30 22:06:47.999446,None,0
4,0fcdf800-d315-465d-8b16-4ba3446f91c8,Paul,r003.1,2025-10-30 22:00:50.701009,None,0
5,d5a1bb75-c218-4b3e-ba53-80e243d15352,Paul,r003.1,2025-10-30 22:00:47.750797,None,0
6,3e91ce2c-72c5-4277-a9d3-d99c92a9ca48,BRad,r003.1,2025-10-30 21:41:37.736409,None,0
7,a2423c19-2ff7-41e2-ae1f-da9e8a296393,Brad,r003.1,2025-10-30 21:35:36.925586,None,0
8,9bee0302-9a8c-4789-8002-c9b9e1bb933d,Brad,r003.1,2025-10-30 21:34:19.621647,None,0
9,69c29155-7973-4c74-a0ef-1d75bf6584d3,BRad,r003.1,2025-10-30 21:27:29.686596,None,0


### Activity Attempts

In [ ]:
# View all activity attempts
attempts = query("""
    SELECT 
        a.attempt_id,
        st.name as student_name,
        a.module,
        a.activity,
        a.score,
        a.total,
        ROUND(a.score * 100.0 / a.total, 1) as percentage,
        a.difficulty,
        a.date
    FROM activity_attempts a
    JOIN students st ON a.student_id = st.student_id
    ORDER BY a.date DESC
""")

print(f"Total Attempts: {len(attempts)}")
display(attempts.head(10))

## 2. Performance Analytics

### Overall Statistics

In [ ]:
# Calculate overall statistics
stats = query("""
    SELECT 
        COUNT(DISTINCT student_id) as total_students,
        COUNT(DISTINCT session_id) as total_sessions,
        COUNT(*) as total_attempts,
        ROUND(AVG(score * 100.0 / total), 1) as avg_percentage,
        ROUND(MIN(score * 100.0 / total), 1) as min_percentage,
        ROUND(MAX(score * 100.0 / total), 1) as max_percentage
    FROM activity_attempts
""")

print("\n📊 Overall Statistics")
print("=" * 50)
display(stats)

### Performance by Student

In [12]:
# Performance by student
student_performance = query("""
    SELECT 
        st.name as student_name,
        COUNT(*) as num_attempts,
        ROUND(AVG(a.score * 100.0 / a.total), 1) as avg_percentage,
        SUM(a.score) as total_correct,
        SUM(a.total) as total_questions,
        MIN(a.date) as first_attempt,
        MAX(a.date) as last_attempt
    FROM activity_attempts a
    JOIN students st ON a.student_id = st.student_id
    GROUP BY st.student_id
    ORDER BY avg_percentage DESC
""")

print("\n👥 Performance by Student")
print("=" * 50)
display(student_performance)


👥 Performance by Student


,student_name,num_attempts,avg_percentage,total_correct,total_questions,first_attempt,last_attempt
0,brad,1,100.0,5,5,2025-10-17 23:16:40.523168,2025-10-17 23:16:40.523168
1,Brad,1,100.0,5,5,2025-10-21 20:55:51.392358,2025-10-21 20:55:51.392358


### Performance by Module

In [ ]:
# Performance by module
module_performance = query("""
    SELECT 
        module,
        activity,
        COUNT(*) as num_attempts,
        ROUND(AVG(score * 100.0 / total), 1) as avg_percentage,
        COUNT(DISTINCT student_id) as num_students
    FROM activity_attempts
    GROUP BY module, activity
    ORDER BY module, activity
""")

print("\n📚 Performance by Module")
print("=" * 50)
display(module_performance)

## 3. Detailed Item Analysis

### Extract Item-Level Results

In [ ]:
# Get all attempts with item results
attempts_with_items = query("""
    SELECT 
        a.attempt_id,
        st.name as student_name,
        a.module,
        a.item_results,
        a.date
    FROM activity_attempts a
    JOIN students st ON a.student_id = st.student_id
    ORDER BY a.date DESC
""")

# Parse JSON item results
item_data = []
for _, row in attempts_with_items.iterrows():
    try:
        items = json.loads(row['item_results'])
        for item in items:
            item_data.append({
                'student_name': row['student_name'],
                'module': row['module'],
                'problem_id': item.get('problem_id'),
                'expression': item.get('expression'),
                'correct_answer': item.get('correct_answer'),
                'student_answer': item.get('student_answer'),
                'correct': item.get('correct'),
                'used_retry': item.get('used_retry', False),
                'date': row['date']
            })
    except:
        pass

items_df = pd.DataFrame(item_data)

if len(items_df) > 0:
    print(f"\n📝 Item-Level Results: {len(items_df)} problems attempted")
    display(items_df.head(10))
else:
    print("No item-level data available yet. Run some quizzes first!")

### Problem Difficulty Analysis

In [ ]:
# Analyze which problems are hardest
if len(items_df) > 0:
    problem_stats = items_df.groupby('expression').agg({
        'correct': ['count', 'sum', 'mean'],
        'used_retry': 'sum'
    }).round(3)
    
    problem_stats.columns = ['attempts', 'correct', 'success_rate', 'retries_used']
    problem_stats = problem_stats.sort_values('success_rate')
    
    print("\n🎯 Problem Difficulty (sorted by success rate)")
    print("=" * 50)
    display(problem_stats)
else:
    print("No problem data available yet.")

## 4. Visualizations

### Score Distribution

In [ ]:
if len(attempts) > 0:
    plt.figure(figsize=(10, 6))
    plt.hist(attempts['percentage'], bins=20, edgecolor='black', alpha=0.7)
    plt.xlabel('Score (%)')
    plt.ylabel('Number of Attempts')
    plt.title('Distribution of Quiz Scores')
    plt.axvline(attempts['percentage'].mean(), color='red', linestyle='--', 
                label=f'Mean: {attempts["percentage"].mean():.1f}%')
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    plt.show()
else:
    print("No data to visualize yet.")

### Performance Over Time

In [ ]:
if len(attempts) > 0:
    # Convert date to datetime
    attempts['date'] = pd.to_datetime(attempts['date'])
    
    # Plot performance over time for each student
    plt.figure(figsize=(12, 6))
    for student in attempts['student_name'].unique():
        student_data = attempts[attempts['student_name'] == student].sort_values('date')
        plt.plot(student_data['date'], student_data['percentage'], 
                marker='o', label=student, linewidth=2)
    
    plt.xlabel('Date')
    plt.ylabel('Score (%)')
    plt.title('Student Performance Over Time')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No data to visualize yet.")

### Problem Success Rates

In [ ]:
if len(items_df) > 0:
    # Calculate success rate by problem
    problem_success = items_df.groupby('expression')['correct'].agg(['sum', 'count'])
    problem_success['success_rate'] = problem_success['sum'] / problem_success['count'] * 100
    problem_success = problem_success.sort_values('success_rate')
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(problem_success)), problem_success['success_rate'])
    plt.yticks(range(len(problem_success)), problem_success.index)
    plt.xlabel('Success Rate (%)')
    plt.title('Success Rate by Problem')
    plt.axvline(50, color='red', linestyle='--', alpha=0.5, label='50%')
    plt.legend()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No problem data available yet.")

## 5. Chat Message Analysis

In [ ]:
# View chat messages
messages = query("""
    SELECT 
        m.message_id,
        st.name as student_name,
        m.agent_type,
        m.sender,
        m.message,
        m.timestamp
    FROM chat_messages m
    JOIN sessions s ON m.session_id = s.session_id
    JOIN students st ON s.student_id = st.student_id
    ORDER BY m.timestamp DESC
    LIMIT 20
""")

if len(messages) > 0:
    print(f"\n💬 Recent Chat Messages: {len(messages)}")
    display(messages)
else:
    print("No chat messages recorded yet.")

## 6. Custom Queries

### Query Template

Use this cell to run your own custom SQL queries:

In [ ]:
# Custom query - modify as needed
custom_query = """
    SELECT * FROM students
    LIMIT 10
"""

result = query(custom_query)
display(result)

## 7. Export Data

In [ ]:
# Export data to CSV files
def export_data():
    """Export all tables to CSV files"""
    export_dir = Path('exports')
    export_dir.mkdir(exist_ok=True)
    
    # Export each table
    tables_to_export = {
        'students': 'SELECT * FROM students',
        'sessions': 'SELECT * FROM sessions',
        'activity_attempts': 'SELECT * FROM activity_attempts',
        'chat_messages': 'SELECT * FROM chat_messages'
    }
    
    for table_name, sql in tables_to_export.items():
        df = query(sql)
        filepath = export_dir / f'{table_name}.csv'
        df.to_csv(filepath, index=False)
        print(f"✓ Exported {table_name} to {filepath}")
    
    print(f"\n✓ All data exported to {export_dir}/")

# Uncomment to export:
# export_data()

## Cleanup

In [ ]:
# Close database connection
conn.close()
print("✓ Database connection closed")